<span style="color:black"><span style = "font-size:30px"> Acquisition of promoter dataset</span>
&nbsp;&nbsp;&nbsp;
   
    
From a previous differential RNA-sequencing (dRNA-seq) dataset, we obtained native promoter sequences of Synechocystis sp. PCC 6803. We assumed the 100-base sequences from a transcription start site (TSS) as a promoter and took normalized reads of the corresponding transcript in the dRNA-seq dataset as the promoter strength. Below is the code description for acquiring the promoter dataset.  

1.	To acquire the promoter dataset from the dRNA-seq dataset, install ‘numpy’ and ‘pandas’ modules, which are modules for handling data arrays and analyzing datasets. 

In [1]:
# 1.

import numpy as np
import pandas as pd

2.	Obtain the whole genome sequence of Synechocystis sp. PCC 6803 in a long string format.

In [2]:
# 2.

pccgene = []
pccseq = ''
data = open('Whole_genome_of_synechocystis.txt','r')
line = data.readlines()
gspoint = line.index('ORIGIN      \')
for i in range(len(line)-gspoint-1):
    templist = line[i+gspoint].strip().split()[1:]
    for partseq in templist:
        pccseq = pccseq+partseq
data.close()
pccseq = pccseq.upper()

SyntaxError: EOL while scanning string literal (<ipython-input-2-0a31bb0702d5>, line 7)

In [3]:
pccgene = []
pccseq = ''

data = open('Whole_genome_of_synechocystis.txt','r')
lines = data.readlines()
line = [line.strip() for line in lines]
pccseq = ''.join(line[1:])
data.close()
pccseq = pccseq.upper()
print(pccseq[:500])

GGCGCGCCATCGCCGGCTGGCGGAAATTACCGAAATGATCCACACCGCTAGTTTGGTCCACGATGACGTGGTGGATGAGGCGGATCTGCGGCGGAATGTGCCCACGGTGAATAGTTTATTTGACAATCGGGTGGCAGTGTTAGCGGGGGATTTCCTCTTTGCCCAATCTTCTTGGTATTTGGCTAACTTAGATAATTTGGAGGTGGTGAAATTATTATCGGAGGTAATTCGGGACTTTGCGGAGGGGGAAATTTTACAGAGCATCAATCGTTTTGACACCGACACAGATTTAGAAACCTATTTGGAAAAAAGCTATTTTAAAACCGCCTCTCTCATTGCCAACAGTGCCAAGGCAGCGGGGGTTTTGAGCGATGCGCCCCGGGATGTGTGTGATCATCTTTACGAATATGGTAAACATTTGGGGTTAGCGTTCCAGATTGTGGACGATATTTTAGATTTCACTTCCCCCACGGAGGTTTTGGGGAAACCGGCCGGGTCAG


In [4]:
len(pccseq)

3573470

3.	Extract the promoter dataset composed of promoter sequences and their strength from the whole genome sequence and the dRNA-seq dataset

In [5]:
!pip install openpyxl

     |████████████████████████████████| 249 kB 3.2 MB/s eta 0:00:01


In [7]:
# 3.

trlist = []
comp_dic = {'A':'T','G':'C','C':'G','T':'A'}
prolen = 100
df = pd.read_excel('Differential_RNA_sequencing_data.xlsx',header = 1,sheet_name = 'S1 - All TUs',engine='openpyxl')
numpydf = df.to_numpy()
print(numpydf[:10])
for i in numpydf:
    # chromosomal DNA
    if i[0] == 'Chr':
        #positive strands
        if i[4] == '+':
            # pccseq[i[2](TSS) - (prolen+1) (TSS로부터 100bp upstream) : TSS-1] - promoter sequence indexing, i[19]: RNA reads
            trlist.append((pccseq[i[2]-(prolen+1):i[2]-1],i[19]))

        # negative strands
        else:
            compseq = ''
            seq = pccseq[i[2]:i[2]+prolen]
            for s in seq:
                compseq +=comp_dic[s]
            #역방향으로 complementary seq promoter 추출
            revcomp_seq = compseq[::-1]
            trlist.append((revcomp_seq, i[19]))
    else:
        break

[['Chr' 'TU1' 105 6 '-' 'a' nan nan nan nan 0 240 133 230 0 198 193 678
  213 106 180 '-N' 678 217.1 3.12 2.83 nan nan nan 'slr0611-2']
 ['Chr' 'TU2' 577 990 '+' 'gai' nan nan 225.0 nan 1 719 100 594 0 938 875
  151 718 643 45 '-Fe' 938 478.3 1.96 1.07 'slr0612'
  'probable pseudouridine synthase' 'TU3' nan]
 ['Chr' 'TU3' 909 810 '-' 'a' nan nan nan nan 0 288 33 326 0 364 205 527
  155 372 45 '-N' 527 231.5 2.28 1.42 nan nan 'TU2' 'slr0612']
 ['Chr' 'TU4' 1532 2150 '+' 'ga' nan nan 45.0 52.0 1 3499 3002 2972 187
  3263 2533 3463 3028 834 1261 '15C' 3499 2404.2 1.46 1.01 'slr0613'
  'hypothetical protein' 'TU5' 'sll0558']
 ['Chr' 'TU6' 2151 2701 '+' 'a' nan nan nan nan 0 1390 500 805 37 1248
  929 2334 1087 1496 360 '-N' 2334 1018.6 2.29 1.56 nan nan 'TU5'
  'sll0558']
 ['Chr' 'TU5' 2900 2061 '-' 'ga' 2899 nan 27.0 111.0 1 2588 601 518 280
  1479 1369 941 1106 802 450 '15C' 2588 1013.4 2.55 1.75 'sll0558'
  'hypothetical protein YCF53' 'TU4, TU6' 'slr0613']
 ['Chr' 'TU7' 3182 2955 '-' '

/NAS/home/yychoi/miniconda3/envs/capstone/lib/python3.6/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


4.	 Export the promoter dataset containing promoter sequences and their strength. This dataset is a training dataset for the VAE and CNN models. 

In [8]:
# 4. 

from openpyxl import Workbook
write_wb = Workbook()
write_ws = write_wb.active

write_ws['A1'] = 'Promoter'
write_ws['B1'] = 'Reads'

for i in range(len(trlist)):
    write_ws.cell(i+2,1,trlist[i][0])
    write_ws.cell(i+2,2,trlist[i][1])

write_wb.save('Training dataset.xlsx')